In [80]:
import numpy as np
import re
from scipy.io import loadmat
import time

1. **Tworzenie macierzy górnotrójkątnej za pomocą eliminacji Gaussa**

In [81]:
A = np.array([[1,2,5],
            [3,8,1],
            [2,-1,4]])
A

array([[ 1,  2,  5],
       [ 3,  8,  1],
       [ 2, -1,  4]])

In [82]:
n = A.shape[0]
for i in range(0,n):
    for j in range(i+1,n):
        e = A[j][i]/A[i][i]
        for k in range(i,n):
            A[j][k] = A[j][k]-e*A[i][k]
A


array([[  1,   2,   5],
       [  0,   2, -14],
       [  0,   0, -41]])

2. **Implementacje metod Jacobiego i Gaussa-Siedla**

In [83]:
def jacobi(A, b, iterations):
    n = b.size
    x = np.zeros(n)
    x_new = np.zeros(n)

    for iter in range(iterations):
        for i in range(n):
            sum = 0
            for j in range(n):
                if(j!=i):
                    sum += A[i][j]*x[j] 
            x_new[i] = 1/A[i][i]*(b[i]-sum)
        x = x_new.copy()

    return x

In [84]:
def gauss_seidl(A, b, iterations):
    n = b.size
    x = np.zeros(n)
    for iter in range(iterations):
        for i in range(n):
            sum = 0
            for j in range(n):
                if(j!=i):
                    sum += A[i][j]*x[j]
            x[i] = 1/A[i][i]*(b[i]-sum)

    return x

3. **Funkcje pomocnicze**

In [85]:
def load_A_b_from_m_file(filename):
    with open(filename, 'r') as f:
        content = f.read()

    content = re.sub(r'%.*', '', content)

    A_match = re.search(r'A\s*=\s*\[(.*?)\];', content, re.DOTALL)
    if not A_match:
        raise ValueError("Nie znaleziono macierzy A")
    A_rows = A_match.group(1).strip().split('\n')
    A = [list(map(float, row.strip().split())) for row in A_rows]

    b_match = re.search(r'b\s*=\s*\[(.*?)\](\'?);', content, re.DOTALL)
    if not b_match:
        raise ValueError("Nie znaleziono wektora b")
    b_data = b_match.group(1).strip().replace('\n', ' ').split()
    b = np.array(list(map(float, b_data)))
    if b_match.group(2) == "'":  # był apostrof, więc wektor pionowy
        b = b.reshape(-1, 1)
    b = b.flatten()
    return np.array(A), b

Funkcje obliczający błąd względny wyniku danej metody względem wyniku dokładnego

In [86]:
def calc_error_jacobi(A,b, iter):
    x = jacobi(A,b, iter)
    x_exact = np.linalg.solve(A,b)
    abs_error = np.linalg.norm(x - x_exact)
    r_error = abs_error/np.linalg.norm(x_exact)

    return r_error

In [87]:
def calc_error_gauss_seidl(A,b, iter):
    x = gauss_seidl(A,b, iter)
    x_exact = np.linalg.solve(A,b)
    abs_error = np.linalg.norm(x - x_exact)
    r_error = abs_error/np.linalg.norm(x_exact)

    return r_error

Funkcje obliczające czas działania danej metody dla danej liczby iteracji


In [88]:
def time_jacobi(A,b, iter):
    start = time.perf_counter()        # czas startu (wysoka rozdzielczość)
    jacobi(A,b,iter)
    end = time.perf_counter()       # czas końca
    result = end - start
    return result

In [89]:
def time_gauss_seidl(A,b, iter):
    start = time.perf_counter()        # czas startu (wysoka rozdzielczość)
    gauss_seidl(A,b,iter)
    end = time.perf_counter()       # czas końca
    result = end - start
    return result